# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [4]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Generate the mathematical model

In [48]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [49]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Food & Beverage Production Planning

**Sets and Indices:**

*  `t`: Index for time periods, `t = 1, ..., 12`
*  `f`: Index for food products, `f = 1, 2`
*  `b`: Index for beverage products, `b = 1, 2, 3`

**Parameters:**

*  `FC_f,t`: Fixed cost of producing one unit of food `f` in period `t` (from "fixed_cost_production.csv")
*  `VC_f,t`: Variable cost of producing one unit of food `f` in period `t` (from "variable_cost_production.csv")
*  `VC_b,t`: Variable cost of producing one liter of beverage `b` in period `t` (from "variable_cost_production.csv")
*  `SC_f,t`: Variable cost of storing one unit of food `f` in period `t` (from "variable_cost_storage.csv")
*  `SC_b,t`: Variable cost of storing one liter of beverage `b` in period `t` (from "variable_cost_storage.csv")
*  `D_f,t`: Demand for food `f` in period `t` (from "demand.csv")
*  `D_b,t`: Demand for beverage `b` in period `t` (from "demand.csv")
*  `R_f,t`: Revenue per unit of food `f` sold in period `t` (from "revenue.csv")
*  `R_b,t`: Revenue per liter of beverage `b` sold in period `t` (from "revenue.csv")
*  `CapS_f`: Storage capacity for food `f` (given as 580, 687)
*  `CapS_b`: Storage capacity for beverage `b` (given as 599, 788, 294)
*  `CapP_f`: Production capacity for food `f` (given as 1080, 908)
*  `CapP_b`: Production capacity for beverage `b` (given as 408, 1000, 403)

**Decision Variables:**

*  `P_f,t`: Quantity of food `f` produced in period `t` (integer)
*  `P_b,t`: Liters of beverage `b` produced in period `t` (continuous)
*  `S_f,t`: Quantity of food `f` stored at the end of period `t` (integer)
*  `S_b,t`: Liters of beverage `b` stored at the end of period `t` (continuous)
*  `Sales_f,t`: Quantity of food `f` sold in period `t` (integer)
*  `Sales_b,t`: Liters of beverage `b` sold in period `t` (continuous)

**Objective Function:**

Maximize total profit over the 12 periods:

```
Maximize:  ∑_(t=1)^12 (∑_(f=1)^2 (R_f,t * Sales_f,t - FC_f,t * (P_f,t > 0) - VC_f,t * P_f,t - SC_f,t * S_f,t) + ∑_(b=1)^3 (R_b,t * Sales_b,t - VC_b,t * P_b,t - SC_b,t * S_b,t)) 
```

**Constraints:**

1. **Production Capacity:** For each product and time period, the production quantity cannot exceed the production capacity:
    *  `P_f,t ≤ CapP_f` for all `f`, `t`
    *  `P_b,t ≤ CapP_b` for all `b`, `t`

2. **Storage Capacity:** For each product and time period, the stored quantity cannot exceed the storage capacity:
    *  `S_f,t ≤ CapS_f` for all `f`, `t`
    *  `S_b,t ≤ CapS_b` for all `b`, `t`

3. **Inventory Balance:** For each product and time period, the stored quantity equals the previous period's storage plus production minus sales:
    *  `S_f,t = S_f,t-1 + P_f,t - Sales_f,t` for all `f`, `t` (with `S_f,0` = 0)
    *  `S_b,t = S_b,t-1 + P_b,t - Sales_b,t` for all `b`, `t` (with `S_b,0` = 0)

4. **Demand Limit:** For each product and time period, the sales quantity cannot exceed the demand:
    *  `Sales_f,t ≤ D_f,t` for all `f`, `t`
    *  `Sales_b,t ≤ D_b,t` for all `b`, `t`

5. **Non-negativity:** All variables must be non-negative:
    *  `P_f,t, P_b,t, S_f,t, S_b,t, Sales_f,t, Sales_b,t ≥ 0` for all `f`, `b`, `t`

This model defines the optimization problem for maximizing profit by planning production and storage of food and beverage products over 12 time periods. The model considers production costs, storage costs, revenue, demand, and capacity constraints. 


## 3. Generate the pyomo code

In [50]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [51]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  
variable_cost_production = {(1, 1): 10, (1, 2): 11, (2, 1): 12, (2, 2): 13,
                          (3, 1): 5, (3, 2): 6, (4, 1): 7, (4, 2): 8} 
variable_cost_storage = {(1, 1): 2, (1, 2): 2.5, (2, 1): 3, (2, 2): 3.5,
                        (3, 1): 1, (3, 2): 1.5, (4, 1): 1.8, (4, 2): 2.2} 
demand = {(1, 1): 500, (1, 2): 550, (2, 1): 600, (2, 2): 650,
          (3, 1): 300, (3, 2): 350, (4, 1): 400, (4, 2): 450} 
revenue = {(1, 1): 20, (1, 2): 22, (2, 1): 24, (2, 2): 26,
           (3, 1): 15, (3, 2): 17, (4, 1): 19, (4, 2): 21}
CapS_f = {1: 580, 2: 687}
CapS_b = {1: 599, 2: 788, 3: 294}
CapP_f = {1: 1080, 2: 908}
CapP_b = {1: 408, 2: 1000, 3: 403}

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.RangeSet(1, 2)  # Food products
model.B = pyo.RangeSet(1, 3)  # Beverage products

# Parameters
model.FC_f = pyo.Param(model.F, model.T, initialize=fixed_cost_production)
model.VC_f = pyo.Param(model.F, model.T, initialize=variable_cost_production)
model.VC_b = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.SC_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage)
model.SC_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)
model.CapS_f = pyo.Param(model.F, initialize=CapS_f)
model.CapS_b = pyo.Param(model.B, initialize=CapS_b)
model.CapP_f = pyo.Param(model.F, initialize=CapP_f)
model.CapP_b = pyo.Param(model.B, initialize=CapP_b)

# Variables
model.P_f = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)  # Food production
model.P_b = pyo.Var(model.B, model.T, within=pyo.NonNegativeReals)  # Beverage production
model.S_f = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)  # Food storage
model.S_b = pyo.Var(model.B, model.T, within=pyo.NonNegativeReals)  # Beverage storage
model.Sales_f = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)  # Food sales
model.Sales_b = pyo.Var(model.B, model.T, within=pyo.NonNegativeReals)  # Beverage sales

# Objective function
def total_profit(model):
    return sum(
        (model.R_f[f, t] * model.Sales_f[f, t] 
         - model.FC_f[f, t] * (model.P_f[f, t] > 0) 
         - model.VC_f[f, t] * model.P_f[f, t]
         - model.SC_f[f, t] * model.S_f[f, t])
        + sum(
            (model.R_b[b, t] * model.Sales_b[b, t]
             - model.VC_b[b, t] * model.P_b[b, t] 
             - model.SC_b[b, t] * model.S_b[b, t])
            for b in model.B)
        for f in model.F for t in model.T
    )

model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# Production capacity
def production_capacity_food(model, f, t):
    return model.P_f[f, t] <= model.CapP_f[f]
model.production_capacity_food = pyo.Constraint(model.F, model.T, rule=production_capacity_food)

def production_capacity_beverage(model, b, t):
    return model.P_b[b, t] <= model.CapP_b[b]
model.production_capacity_beverage = pyo.Constraint(model.B, model.T, rule=production_capacity_beverage)

# Storage capacity
def storage_capacity_food(model, f, t):
    return model.S_f[f, t] <= model.CapS_f[f]
model.storage_capacity_food = pyo.Constraint(model.F, model.T, rule=storage_capacity_food)

def storage_capacity_beverage(model, b, t):
    return model.S_b[b, t] <= model.CapS_b[b]
model.storage_capacity_beverage = pyo.Constraint(model.B, model.T, rule=storage_capacity_beverage)

# Inventory balance
def inventory_balance_food(model, f, t):
    if t == 1:
        return model.S_f[f, t] == model.P_f[f, t] - model.Sales_f[f, t]
    else:
        return model.S_f[f, t] == model.S_f[f, t-1] + model.P_f[f, t] - model.Sales_f[f, t]
model.inventory_balance_food = pyo.Constraint(model.F, model.T, rule=inventory_balance_food)

def inventory_balance_beverage(model, b, t):
    if t == 1:
        return model.S_b[b, t] == model.P_b[b, t] - model.Sales_b[b, t]
    else:
        return model.S_b[b, t] == model.S_b[b, t-1] + model.P_b[b, t] - model.Sales_b[b, t]
model.inventory_balance_beverage = pyo.Constraint(model.B, model.T, rule=inventory_balance_beverage)

# Demand limit
def demand_limit_food(model, f, t):
    return model.Sales_f[f, t] <= model.D_f[f, t]
model.demand_limit_food = pyo.Constraint(model.F, model.T, rule=demand_limit_food)

def demand_limit_beverage(model, b, t):
    return model.Sales_b[b, t] <= model.D_b[b, t]
model.demand_limit_beverage = pyo.Constraint(model.B, model.T, rule=demand_limit_beverage)

# Solve
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Display results
print(results)
print("Optimal Profit:", pyo.value(model.profit))
```

**Notes:**

*  Replace the sample data provided with actual data from your CSV files.
*  The `(model.P_f[f, t] > 0)` expression in the objective function is a logical condition that evaluates to 1 if production is greater than 0, triggering the fixed cost. 
*  Choose an appropriate solver (`cbc`, `gurobi`, `cplex`, etc.) based on availability and licensing. 
*  The output of this code will show the solver status (optimal, infeasible, etc.) and the optimal objective function value (total profit). You can also access the values of the decision variables (production, storage, sales) to see the optimal production plan.


## 4. Input problem data and try running the generated code

In [52]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  
variable_cost_production = {(1, 1): 10, (1, 2): 11, (2, 1): 12, (2, 2): 13,
                          (3, 1): 5, (3, 2): 6, (4, 1): 7, (4, 2): 8} 
variable_cost_storage = {(1, 1): 2, (1, 2): 2.5, (2, 1): 3, (2, 2): 3.5,
                        (3, 1): 1, (3, 2): 1.5, (4, 1): 1.8, (4, 2): 2.2} 
demand = {(1, 1): 500, (1, 2): 550, (2, 1): 600, (2, 2): 650,
          (3, 1): 300, (3, 2): 350, (4, 1): 400, (4, 2): 450} 
revenue = {(1, 1): 20, (1, 2): 22, (2, 1): 24, (2, 2): 26,
           (3, 1): 15, (3, 2): 17, (4, 1): 19, (4, 2): 21}
CapS_f = {1: 580, 2: 687}
CapS_b = {1: 599, 2: 788, 3: 294}
CapP_f = {1: 1080, 2: 908}
CapP_b = {1: 408, 2: 1000, 3: 403}

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.RangeSet(1, 2)  # Food products
model.B = pyo.RangeSet(1, 3)  # Beverage products

# Parameters
model.FC_f = pyo.Param(model.F, model.T, initialize=fixed_cost_production)
model.VC_f = pyo.Param(model.F, model.T, initialize=variable_cost_production)
model.VC_b = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.SC_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage)
model.SC_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)
model.CapS_f = pyo.Param(model.F, initialize=CapS_f)
model.CapS_b = pyo.Param(model.B, initialize=CapS_b)
model.CapP_f = pyo.Param(model.F, initialize=CapP_f)
model.CapP_b = pyo.Param(model.B, initialize=CapP_b)

# Variables
model.P_f = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)  # Food production
model.P_b = pyo.Var(model.B, model.T, within=pyo.NonNegativeReals)  # Beverage production
model.S_f = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)  # Food storage
model.S_b = pyo.Var(model.B, model.T, within=pyo.NonNegativeReals)  # Beverage storage
model.Sales_f = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)  # Food sales
model.Sales_b = pyo.Var(model.B, model.T, within=pyo.NonNegativeReals)  # Beverage sales

# Objective function
def total_profit(model):
    return sum(
        (model.R_f[f, t] * model.Sales_f[f, t] 
         - model.FC_f[f, t] * (model.P_f[f, t] > 0) 
         - model.VC_f[f, t] * model.P_f[f, t]
         - model.SC_f[f, t] * model.S_f[f, t])
        + sum(
            (model.R_b[b, t] * model.Sales_b[b, t]
             - model.VC_b[b, t] * model.P_b[b, t] 
             - model.SC_b[b, t] * model.S_b[b, t])
            for b in model.B)
        for f in model.F for t in model.T
    )

model.profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

# Constraints
# Production capacity
def production_capacity_food(model, f, t):
    return model.P_f[f, t] <= model.CapP_f[f]
model.production_capacity_food = pyo.Constraint(model.F, model.T, rule=production_capacity_food)

def production_capacity_beverage(model, b, t):
    return model.P_b[b, t] <= model.CapP_b[b]
model.production_capacity_beverage = pyo.Constraint(model.B, model.T, rule=production_capacity_beverage)

# Storage capacity
def storage_capacity_food(model, f, t):
    return model.S_f[f, t] <= model.CapS_f[f]
model.storage_capacity_food = pyo.Constraint(model.F, model.T, rule=storage_capacity_food)

def storage_capacity_beverage(model, b, t):
    return model.S_b[b, t] <= model.CapS_b[b]
model.storage_capacity_beverage = pyo.Constraint(model.B, model.T, rule=storage_capacity_beverage)

# Inventory balance
def inventory_balance_food(model, f, t):
    if t == 1:
        return model.S_f[f, t] == model.P_f[f, t] - model.Sales_f[f, t]
    else:
        return model.S_f[f, t] == model.S_f[f, t-1] + model.P_f[f, t] - model.Sales_f[f, t]
model.inventory_balance_food = pyo.Constraint(model.F, model.T, rule=inventory_balance_food)

def inventory_balance_beverage(model, b, t):
    if t == 1:
        return model.S_b[b, t] == model.P_b[b, t] - model.Sales_b[b, t]
    else:
        return model.S_b[b, t] == model.S_b[b, t-1] + model.P_b[b, t] - model.Sales_b[b, t]
model.inventory_balance_beverage = pyo.Constraint(model.B, model.T, rule=inventory_balance_beverage)

# Demand limit
def demand_limit_food(model, f, t):
    return model.Sales_f[f, t] <= model.D_f[f, t]
model.demand_limit_food = pyo.Constraint(model.F, model.T, rule=demand_limit_food)

def demand_limit_beverage(model, b, t):
    return model.Sales_b[b, t] <= model.D_b[b, t]
model.demand_limit_beverage = pyo.Constraint(model.B, model.T, rule=demand_limit_beverage)

# Solve
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Display results
print(results)
print("Optimal Profit:", pyo.value(model.profit))

ERROR: Rule failed for Param 'VC_f' with index (3, 1): KeyError: "Index '(3,
1)' is not valid for indexed component 'VC_f'"
ERROR: Constructing component 'VC_f' from data=None failed:
        KeyError: "Index '(3, 1)' is not valid for indexed component 'VC_f'"


KeyError: "Index '(3, 1)' is not valid for indexed component 'VC_f'"

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [53]:
print(response.text)

## Mathematical Optimization Model for Food & Beverage Production Planning

**Sets and Indices:**

*  `t`: Index for time periods, `t = 1, ..., 12`
*  `f`: Index for food products, `f = 1, 2`
*  `b`: Index for beverage products, `b = 1, 2, 3`

**Parameters:**

*  `FC_f,t`: Fixed cost of producing one unit of food `f` in period `t` (from "fixed_cost_production.csv")
*  `VC_f,t`: Variable cost of producing one unit of food `f` in period `t` (from "variable_cost_production.csv")
*  `VC_b,t`: Variable cost of producing one liter of beverage `b` in period `t` (from "variable_cost_production.csv")
*  `SC_f,t`: Variable cost of storing one unit of food `f` in period `t` (from "variable_cost_storage.csv")
*  `SC_b,t`: Variable cost of storing one liter of beverage `b` in period `t` (from "variable_cost_storage.csv")
*  `D_f,t`: Demand for food `f` in period `t` (from "demand.csv")
*  `D_b,t`: Demand for beverage `b` in period `t` (from "demand.csv")
*  `R_f,t`: Revenue per unit of food `f` sol

In [54]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  
variable_cost_production = {(1, 1): 10, (1, 2): 11, (2, 1): 12, (2, 2): 13,
                          (3, 1): 5, (3, 2): 6, (4, 1): 7, (4, 2): 8} 
variable_cost_storage = {(1, 1): 2, (1, 2): 2.5, (2, 1): 3, (2, 2): 3.5,
                        (3, 1): 1, (3, 2): 1.5, (4, 1): 1.8, (4, 2): 2.2} 
demand = {(1, 1): 500, (1, 2): 550, (2, 1): 600, (2, 2): 650,
          (3, 1): 300, (3, 2): 350, (4, 1): 400, (4, 2): 450} 
revenue = {(1, 1): 20, (1, 2): 22, (2, 1): 24, (2, 2): 26,
           (3, 1): 15, (3, 2): 17, (4, 1): 19, (4, 2): 21}
CapS_f = {1: 580, 2: 687}
CapS_b = {1: 599, 2: 788, 3: 294}
CapP_f = {1: 1080, 2: 908}
CapP_b = {1: 408, 2: 1000, 3: 403}

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.RangeSet(1, 2)  # Food products
model.B = pyo.RangeSet(1, 3) 